# Plot failure domain for toy problems

In [8]:
from os import path
import rareeventestimation as ree
import numpy as np
import plotly.express as px
from rareeventestimation.evaluation.constants import INDICATOR_APPROX_LATEX_NAME, BM_SOLVER_SCATTER_STYLE, MY_LAYOUT, DF_COLUMNS_TO_LATEX, LATEX_TO_HTML, WRITE_SCALE, WIDTH
import plotly.graph_objects as go
from IPython.display import display, Markdown
# recommended: use autoreload for development: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# problem and solver stuff
import plotly.io as pio
cvar_tgt= 1
plot_fitted_enkf_sample = False  
sample_size = 1000
problem_list = [ree.prob_convex]
methods = [ree.CBREE(seed=1, cvar_tgt=cvar_tgt, divergence_check=False),  ree.ENKF(seed=1, cvar_tgt=cvar_tgt)]
marker_shape_list = ["circle", "square", "cross-open"]
#figure stuff
safe_annotation_anchor = [0, 0]
failure_annotation_anchor = [4 , 4]
delta = 0.1
x0 = -1
x1 = 6
xx = np.arange(x0,x1, delta)
yy = np.arange(x0,x1, delta)
col_scale = [[0, "lightgrey"], [1, "white"]]
contour_style = {"start": 0, "end": 0, "size": 0, "showlabels": False}
for i, prob in enumerate(problem_list):
    fig = go.Figure()
    # contour plot
    zz_lsf = np.zeros((len(yy), len(xx)))
    for (xi, x) in enumerate(xx):
        for(yi, y) in enumerate(yy):
            z = np.array([x, y])
            zz_lsf[yi, xi] = prob.lsf(z)
    c_lsf = go.Contour(z=zz_lsf, x=xx, y=yy, colorscale=col_scale,
                        contours=contour_style, line_width=2, showscale=False, showlegend=False)
    fig.add_trace(c_lsf)
    
    # scatter
    for j, solver in enumerate(methods):
        prob.set_sample(sample_size, seed=1)
        sol = solver.solve(prob)
        normal_sample = (plot_fitted_enkf_sample and str(solver) == "EnKF (GM)") or str(solver) != "EnKF (GM)"
        xx = sol.ensemble_hist[-1,:,0] if normal_sample else sol.other["Final Iteration"][-1,:,0] 
        yy = sol.ensemble_hist[-1,:,1] if normal_sample else sol.other["Final Iteration"][-1,:,1] 
        sc = go.Scatter(
            x = xx,
            y= yy,
            name= str(solver),
            mode="markers",
            opacity=1,
            marker_symbol = marker_shape_list[j],
        )
        fig.add_trace(sc)
    # style
    fig.update_layout(**MY_LAYOUT)
    fig.update_layout(height=SIAM_TEXTWIDTH_PX,
                      width=SIAM_TEXTWIDTH_PX,
                      legend_orientation="h",
                      xaxis_range = [x0, x1],
                      yaxis_range = [x0, x1])
    fig.add_annotation(x=failure_annotation_anchor[0],
                       y=failure_annotation_anchor[1],
                       ay=0,
                       ax=0,
                       text="{<i>G</i> \u2264 0}")
    fig.add_annotation(x=safe_annotation_anchor[0],
                       y=safe_annotation_anchor[1],
                       ay=0,
                       ax=0,
                       text="{<i>G</i> > 0}")
    # save and show figure
    fig_name = f"{prob.name} scatter plot{'' if plot_fitted_enkf_sample else ' no enkf fit '}".replace(" ", "_").lower()
    fig.write_image(fig_name +".pdf")
    fig.show()
    # make and save caption
    fig_description = f"Failure domain of the {prob.name}. \
The final ensembles of the EnKF and CBREE methods applied to the convex limit state function $G(x)= \\frac{{(x_1-x_2)^2}}{{10}} - \\frac{{x_1 +x_2}}{{\sqrt{{2}}}} + \\frac{{2}}{{5}}$. " +\
("" if plot_fitted_enkf_sample else "Note that for the EnKF method this is not the sample fitted to last particle ensemble of the internal iteration, which is  used for importance sampling. ") + \
f"Each method used $J={sample_size}$ samples and the stopping criterion $\\Delta_{{\\text{{Target}}}}$ = {cvar_tgt}. \
The CBREE method performed no divergence check, used the approximation $I_\\text{{alg}}$, the stepsize control $\\epsilon_{{\\text{{Target}}}}=0.5$ and controled the increase of $\\sigma$ with $\\text{{Lip}}(\\varsigma) = 1$."
    with open(fig_name + "_desc.tex", "w") as file:
        file.write(fig_description)
    display(Markdown(fig_description))

Failure domain of the Convex Problem. The final ensembles of the EnKF and CBREE methods applied to the convex limit state function $G(x)= \frac{(x_1-x_2)^2}{10} - \frac{x_1 +x_2}{\sqrt{2}} + \frac{2}{5}$. Note that for the EnKF method this is not the sample fitted to last particle ensemble of the internal iteration, which is  used for importance sampling. Each method used $J=1000$ samples and the stopping criterion $\Delta_{\text{Target}}$ = 1. The CBREE method performed no divergence check, used the approximation $I_\text{alg}$, the stepsize control $\epsilon_{\text{Target}}=0.5$ and controled the increase of $\sigma$ with $\text{Lip}(\varsigma) = 1$.